In [1]:
# Import the needed packages
import numpy as np
import pandas as pd
import bokeh.plotting as bk
import bokeh.io as bi
from bokeh.io import export_svgs
#bi.output_notebook()

# TUBS Rot, Gelb, Orange, Hellgrün, Grün, Dunkelgrün, Hellblau, Blau, Dunkelblau, Violett
TUBScolorscale=[(190,30,60,0.9),
               (255,200,42,0.9),
               (225,109,0,0.9),
               (172,193,58,0.9),
               (109,131,0,0.9),
               (0,83,74,0.9),
               (102,180,211,0.9),
               (0,112,155,0.9),
               (0,63,87,0.9),
               (138,48,127,0.9)]

In [3]:
# Open the data file
# Read the Data
Data = pd.read_csv('./Data/H01_TIRAND.csv', sep=';')
# Give the Columns
Data.columns

Index(['Unnamed: 0', 'Degree', 'MS_RGA', 'w_MS_RGA', 'Grad_RGA', 'MS_A',
       'w_MS_A', 'Grad_A', 'MS_D', 'w_MS_D', 'Grad_D', 'w_0.01_RGA',
       'w_0.01_A', 'w_0.01_D', 'w_0.02_RGA', 'w_0.02_A', 'w_0.02_D',
       'w_0.1_RGA', 'w_0.1_A', 'w_0.1_D', 'w_0.00666666666667_RGA',
       'w_0.00666666666667_A', 'w_0.00666666666667_D', 'w_0.005_RGA',
       'w_0.005_A', 'w_0.005_D'],
      dtype='object')

In [3]:
# Get the PDF and CDF of the Maximum Sensitivity and the frequency and plot
from scipy.stats import norm
from scipy.signal import savgol_filter
from bokeh.models import BoxAnnotation

# Import the data
sample_data = Data[Data['MS_RGA']>0.0]['MS_RGA']
# Get the maximum sensitivity limits
x = np.linspace(min(sample_data),max(sample_data), len(sample_data))
# Calculate the PDF and CDF
pdf = norm.pdf(x,sample_data)
cdf = norm.cdf(x,sample_data)
# Smooth
pdf_s = savgol_filter(pdf, 61, 3)
cdf_s = savgol_filter(cdf, 101, 3)
# Make the MS=1.4 Plot
MS = [1.4,1.4]
YMS = [0.0, 1.0]
# Plot
p1 = bk.figure(x_axis_type = "log", x_axis_label = "Maximum Singular Values", 
               y_axis_label = "Propability", width = 800, height=300,
               x_range = [9e-1,1e2], y_range = [0,1.1])
# Add the robustness and performance limits
p1.add_layout(BoxAnnotation(left=1.4, fill_alpha=0.1, fill_color='red'))
p1.add_layout(BoxAnnotation(left = 1.0,right=1.4, fill_alpha=0.1, fill_color='green'))
p1.add_layout(BoxAnnotation(right=1.0, fill_alpha=0.1, fill_color='yellow'))
# Add the Data
p1.scatter(x, pdf, color = TUBScolorscale[6], legend = "PDF", alpha = 0.7)
p1.line(x, pdf_s, color = TUBScolorscale[6])
p1.scatter(x,cdf, color = TUBScolorscale[2], legend = "CDF", alpha = 0.7, marker = "cross")
p1.line(x, cdf_s, color = TUBScolorscale[2])
# Add the legend
p1.legend.location="top_left"
# Output and Save
p1.output_backend = "svg"
export_svgs(p1, filename="MIMO_H01_TIRAND_PDF_MS_RGA.svg")
#bk.show(p1)

['MIMO_H01_TIRAND_PDF_MS_RGA.svg']

In [4]:
# Get the PDF and CDF of the Maximum Sensitivity

# Import the data
sample_data = Data[Data['MS_A']>0.0]['MS_A']
# Get the maximum sensitivity limits
x = np.linspace(min(sample_data),max(sample_data), len(sample_data))
# Calculate the PDF and CDF
pdf = norm.pdf(x,sample_data)
cdf = norm.cdf(x,sample_data)
# Smooth
pdf_s = savgol_filter(pdf, 61, 3)
cdf_s = savgol_filter(cdf, 101, 3)

# Plot
p1 = bk.figure(x_axis_type = "log", x_axis_label = "Maximum Singular Values", 
               y_axis_label = "Propability", width = 800, height=300,
               x_range = [9e-1,1e2], y_range = [0,1.1])
# Get the robustness and performance limits
p1.add_layout(BoxAnnotation(left=1.4, fill_alpha=0.1, fill_color='red'))
p1.add_layout(BoxAnnotation(left = 1.0,right=1.4, fill_alpha=0.1, fill_color='green'))
p1.add_layout(BoxAnnotation(right=1.0, fill_alpha=0.1, fill_color='yellow'))
# Add the Data
p1.scatter(x, pdf, color = TUBScolorscale[6], legend = "PDF", alpha = 0.7)
p1.line(x, pdf_s, color = TUBScolorscale[6])
p1.scatter(x,cdf, color = TUBScolorscale[2], legend = "CDF", alpha = 0.7, marker = "cross")
p1.line(x, cdf_s, color = TUBScolorscale[2])
# Add the legend
p1.legend.location="top_left"
bk.show(p1)
# Output and Save
p1.output_backend = "svg"
export_svgs(p1, filename="MIMO_H01_TIRAND_PDF_MS_A.svg")

['MIMO_H01_TIRAND_PDF_MS_A.svg']

In [5]:
# Get the PDF and CDF of the Maximum Sensitivity and the frequency

# Import the data
sample_data = Data[Data['MS_D']>0.0]['MS_D']
# Get the maximum sensitivity limits
x = np.linspace(min(sample_data),max(sample_data), len(sample_data))
# Calculate the PDF and CDF
pdf = norm.pdf(x,sample_data)
cdf = norm.cdf(x,sample_data)
# Smooth
pdf_s = savgol_filter(pdf, 61, 3)
cdf_s = savgol_filter(cdf, 101, 3)

# Plot
p1 = bk.figure(x_axis_type = "log", x_axis_label = "Maximum Singular Values", 
               y_axis_label = "Propability", width = 800, height=300,
               x_range = [9e-1,1e2], y_range = [0,1.1])
# Get the robustness and performance limits
p1.add_layout(BoxAnnotation(left=1.4, fill_alpha=0.1, fill_color='red'))
p1.add_layout(BoxAnnotation(left = 1.0,right=1.4, fill_alpha=0.1, fill_color='green'))
p1.add_layout(BoxAnnotation(right=1.0, fill_alpha=0.1, fill_color='yellow'))
# Add the Data
p1.scatter(x, pdf, color = TUBScolorscale[6], legend = "PDF", alpha = 0.7)
p1.line(x, pdf_s, color = TUBScolorscale[6])
p1.scatter(x,cdf, color = TUBScolorscale[2], legend = "CDF", alpha = 0.7, marker = "cross")
p1.line(x, cdf_s, color = TUBScolorscale[2])
# Add the legend
p1.legend.location="top_left"
# Output and Save
p1.output_backend = "svg"
export_svgs(p1, filename="MIMO_H01_TIRAND_PDF_MS_D.svg")
#bk.show(p1)

['MIMO_H01_TIRAND_PDF_MS_D.svg']

In [26]:
# Get the PDF and CDF of the frequency
# DEPENDS ON THE STUDY!!!
# PTN -> T = 80...180
# TSUM -> T = 180

# Import the data
sample_data = Data[Data['w_MS_RGA']>0.0]['w_MS_RGA']
# Get the maximum sensitivity limits
x = np.linspace(0,max(sample_data), len(sample_data))
# Calculate the PDF and CDF
pdf = norm.pdf(x,sample_data)
cdf = norm.cdf(x,sample_data)
# Smooth
pdf_s = savgol_filter(pdf, 61, 7)
cdf_s = savgol_filter(cdf, 101, 7)

# Plot
p1 = bk.figure(x_axis_type = "log", x_axis_label = "Frequency of Maximum Singular Value [rad/s]",
               y_axis_label = "Propability", width = 800, height = 300,
              x_range = [1e-1,1e2], y_range = [0,1.1])
# Get the robustness and performance limits
p1.add_layout(BoxAnnotation(left=1.0/180.0, fill_alpha=0.1, fill_color='red'))
p1.add_layout(BoxAnnotation(right=1.0/180.0, left = 3.0 / 180.0, fill_alpha=0.1, fill_color='green'))
p1.add_layout(BoxAnnotation(right=3.0 / 180.0, fill_alpha=0.1, fill_color='yellow'))
# Add the Data
p1.scatter(x, pdf, color = TUBScolorscale[6], legend = "PDF", alpha = 0.7)
p1.line(x, pdf_s, color = TUBScolorscale[6])
p1.scatter(x,cdf, color = TUBScolorscale[2], legend = "CDF", alpha = 0.7, marker = "cross")
p1.line(x, cdf_s, color = TUBScolorscale[2])
# Add the legend
p1.legend.location="top_left"
# Output and Save
#p1.output_backend = "svg"
#export_svgs(p1, filename="MIMO_H01_TIRAND_PDF_w_MS_RGA.svg")
bk.show(p1)

In [27]:
# Get the PDF and CDF of the frequency
# DEPENDS ON THE STUDY!!!
# PTN -> T = 80...180
# TSUM -> T = 180

# Import the data
sample_data = Data[Data['w_MS_A']>0.0]['w_MS_A']
# Get the maximum sensitivity limits
x = np.linspace(0,max(sample_data), len(sample_data))
# Calculate the PDF and CDF
pdf = norm.pdf(x,sample_data)
cdf = norm.cdf(x,sample_data)
# Smooth
pdf_s = savgol_filter(pdf, 61, 7)
cdf_s = savgol_filter(cdf, 101, 7)

# Plot
p1 = bk.figure(x_axis_type = "log", x_axis_label = "Frequency of Maximum Singular Value [rad/s]",
               y_axis_label = "Propability", width = 800, height = 300,
              x_range = [1e-1,1e2], y_range = [0,1.1])
# Get the robustness and performance limits
p1.add_layout(BoxAnnotation(left=1.0/180.0, fill_alpha=0.1, fill_color='red'))
p1.add_layout(BoxAnnotation(right=1.0/180.0, left = 3.0 / 180.0, fill_alpha=0.1, fill_color='green'))
p1.add_layout(BoxAnnotation(right=3.0 / 180.0, fill_alpha=0.1, fill_color='yellow'))
# Add the Data
p1.scatter(x, pdf, color = TUBScolorscale[6], legend = "PDF", alpha = 0.7)
p1.line(x, pdf_s, color = TUBScolorscale[6])
p1.scatter(x,cdf, color = TUBScolorscale[2], legend = "CDF", alpha = 0.7, marker = "cross")
p1.line(x, cdf_s, color = TUBScolorscale[2])
# Add the legend
p1.legend.location="top_left"
# Output and Save
#p1.output_backend = "svg"
#export_svgs(p1, filename="MIMO_H01_TIRAND_PDF_w_MS_A.svg")
bk.show(p1)

In [4]:
# Get the PDF and CDF of the frequency
# DEPENDS ON THE STUDY!!!
# PTN -> T = 80...180
# TSUM -> T = 180
from scipy.stats import norm
from scipy.signal import savgol_filter
from bokeh.models import BoxAnnotation

# Import the data
sample_data = Data[Data['w_MS_D']>1e-3]['w_MS_D']
# Get the maximum sensitivity limits
x = np.linspace(min(sample_data),max(sample_data), len(sample_data))
# Calculate the PDF and CDF
pdf = norm.pdf(x,sample_data)
cdf = norm.cdf(x,sample_data)

# Smooth
pdf_s = savgol_filter(pdf, 101, 11)
cdf_s = savgol_filter(cdf, 101, 8)

# Plot
p1 = bk.figure(x_axis_type = "log", x_axis_label = "Frequency of Maximum Singular Value [rad/s]",
               y_axis_label = "Propability", width = 800, height = 300,
               x_range = [1e-1,1e2], y_range = [-.2,1.2])

# Get the robustness and performance limits
p1.add_layout(BoxAnnotation(left=1.0/180.0, fill_alpha=0.1, fill_color='red'))
p1.add_layout(BoxAnnotation(right=1.0/180.0, left = 3.0 / 180.0, fill_alpha=0.1, fill_color='green'))
p1.add_layout(BoxAnnotation(right=3.0 / 180.0, fill_alpha=0.1, fill_color='yellow'))
# Add the Data
p1.scatter(x, pdf, color = TUBScolorscale[6], legend = "PDF", alpha = 0.7)
p1.line(x, pdf_s, color = TUBScolorscale[6])
p1.scatter(x,cdf, color = TUBScolorscale[2], legend = "CDF", alpha = 0.7, marker = "cross")
p1.line(x, cdf_s, color = TUBScolorscale[2])
# Add the legend
p1.legend.location="top_left"
# Output and Save
#p1.output_backend = "svg"
#export_svgs(p1, filename="MIMO_H01_TIRAND_PDF_w_MS_D.svg")
bk.show(p1)

In [9]:
# Other Plot
sample_data = Data[Data['MS_RGA']>0.0]['MS_RGA']
samples = range(0,len(sample_data))
p2 = bk.figure(title="Maximum Singular Values per Sample", x_axis_label = "Sample No.", y_axis_label="Maximum Singular Value",height= 800, width = 800, y_axis_type = "log")
p2.scatter(samples,sample_data, color = TUBScolorscale[6], marker = "circle", legend = "RGA")
sample_data = Data[Data['MS_A']>0.0]['MS_A']
samples = range(0,len(sample_data))
p2.scatter(samples,sample_data, color = TUBScolorscale[0], marker = "cross", legend = "Aström Decoupled")
sample_data = Data[Data['MS_D']>0.0]['MS_D']
samples = range(0,len(sample_data))
p2.scatter(samples,sample_data, color = TUBScolorscale[2], marker = "diamond", legend = "Modified Aström")


# Show and Export
#p2.output_backend = "svg"
#export_svgs(p2, filename="MIMO_H01_TIRAND_MS_OVER_SAMPLES_ALL.svg")
#bk.show(p2)

GlyphRenderer(id='ee613373-a8a6-4aa9-ae89-39b2b159dca0', ...)

In [10]:
# Initialize BIG Plot

# Scatter
p1 = bk.figure(x_axis_type = "log", x_axis_label ="Frequency of Maximum Singular Value", y_axis_label = "Maximum Singular Value",height= 600, width = 600,  y_axis_type = "log")

# Collect Data
sample_data = Data[Data['MS_RGA']>0.0]['MS_RGA']
samples = Data[Data['MS_RGA']>0.0]['w_MS_RGA']
# Add Scatter
p1.scatter(samples,sample_data, color = TUBScolorscale[6], marker = "circle", legend = "RGA")
# Collect Data
sample_data = Data[Data['MS_A']>0.0]['MS_A']
samples = Data[Data['MS_RGA']>0.0]['w_MS_A']
# Add Scatter
p1.scatter(samples,sample_data, color = TUBScolorscale[0],  marker = "cross", legend = "Aström Decoupled")
# Collect Data
sample_data = Data[Data['MS_D']>0.0]['MS_D']
samples = Data[Data['MS_RGA']>0.0]['w_MS_D']
# Add Scatter
p1.scatter(samples,sample_data, color = TUBScolorscale[2], marker = "diamond", legend = "Modified Aström")

bk.show(p1)

# Show and Export
p1.output_backend = "svg"
export_svgs(p2, filename="MIMO_H01_TIRAND_MS_OVER_Frequency_ALL.svg")

In [11]:
# Get the histogram of MS_RGA

from bokeh.layouts import gridplot, layout

sample_data = Data[['MS_RGA']]
hist, edges = np.histogram(sample_data, density = True, bins = 1000, range=(1,2))
p1 = bk.figure(title="Distribution of Maximum Singular Values", y_axis_label = "Samples",height= 300, width = 800)
p1.quad(top=hist, bottom = 0, left=edges[:-1], right=edges[1:], alpha = 1, color = TUBScolorscale[6])


sample_data = Data[['MS_A']]
hist, edges = np.histogram(sample_data, density = True, bins = 1000, range=(1,2))
p2 = bk.figure(y_axis_label = "Samples",height= 300, width = 800)
p2.quad(top=hist, bottom = 0, left=edges[:-1], right=edges[1:], alpha = 1, color = TUBScolorscale[0])

sample_data = Data[['MS_D']]
hist, edges = np.histogram(sample_data, density = True, bins = 1000, range=(1,2))
p3 = bk.figure(y_axis_label = "Samples", x_axis_label = "Maximum Singular Value",height= 300, width = 800)
p3.quad(top=hist, bottom = 0, left=edges[:-1], right=edges[1:], alpha = 1, color = TUBScolorscale[2])

# Show and Export
p1.output_backend = "svg"
export_svgs(p1, filename="MIMO_H01_TIRAND_HIST_MS_RGA.svg")

p2.output_backend = "svg"
export_svgs(p2, filename="MIMO_H01_TIRAND_HIST_MS_A.svg")

p3.output_backend = "svg"
export_svgs(p3, filename="MIMO_H01_TIRAND_HIST_MS_D.svg")


#bk.show(p)

['MIMO_H01_TIRAND_HIST_MS_D.svg']